In [17]:
using JuMP
using Ipopt
using MadNLP


GU = Dict(:pi => 0.01, 
          :L => 0.5, 
          :sigma => 0.9, 
          :kvalue => 0, 
          :nfirm => 1, 
          :rho => 0, 
          :gammamax => 0)

GU[:rho] = 1-1/GU[:sigma]
GU[:gammamax] = GU[:pi]*(1-GU[:L])^(-1/GU[:sigma]) / (GU[:pi] * (1-GU[:L])^(-1/GU[:sigma]) + 1 - GU[:pi])


function nash(GU)
    pi = GU[:pi]
    L = GU[:L]
    σ = GU[:sigma]
    kvalue = GU[:kvalue]
    nfirm = GU[:nfirm]
    ρ = GU[:rho]
    gammamax = GU[:gammamax]


    m = Model()

    @variables(m,begin
         P_G>=1e-5, (start = 1-pi,)	#Price index for consumption on a good day,
		 P_B>=1e-5, (start = pi,)	#Price index for consumption on a bad day,
		 C_G, (start = 1-pi + pi*(1-L),)	#Consumption in the good day,
		 C_B, (start = 1-pi + pi*(1-L),)	#Consumption in the bad day,
		 P_C>=1e-5, (start = 1,)	#Consumption price index,
		 M, (start = 1-pi + pi*(1-L),)	#Income,
		 K>=0, (start = (1-pi + pi*(1-L) - (1-L))/(2*(1-pi)),)	#Coverage
		0<=GAMMA<=gammamax, (start = gammamax/2,)	#Premium for coverage
    end)

    fix(P_G,1-pi,force=true)

    @NLobjective(m,Max,(GAMMA-pi)*(K-kvalue*(nfirm-1)/nfirm))

    @NLconstraints(m,begin
        income, M == P_G + P_B*(1-L)
        P_Cdef, P_C == ( (1-pi) * (P_G/(1-pi))^(1-σ) + pi * (P_B/pi)^(1-σ) )^(1/(1-σ))
        C_Gdef, C_G == M/P_C * (P_C*(1-pi)/P_G)^σ
        C_Bdef, C_B == M/P_C * (P_C*pi/P_B)^σ
        market_G, 1 == C_G + GAMMA*K
        market_B, 1 - L + K == C_B + GAMMA*K
        demand_K, GAMMA*(P_G+P_B) == P_B
    end)


    return m
end

nash (generic function with 2 methods)

In [18]:
GU[:nfirm] = 2
GU[:kvalue] = .5

Nash_ipopt = nash(GU)

solver = optimizer_with_attributes(Ipopt.Optimizer, "tol" => 1e-12)

set_optimizer(Nash_ipopt,solver)
set_silent(Nash_ipopt)

#set_attribute(Nash_ipopt,"tol",1e-10)

optimize!(Nash_ipopt)
obj_ipopt = objective_value(Nash_ipopt)

#Nash_mad = nash(GU,MadNLP.Optimizer)
#set_silent(Nash_mad)
#optimize!(Nash_mad)
#obj_mad = objective_value(Nash_mad)

#println("Ipopt Objective: $(obj_ipopt)\nMad Objective: $(obj_mad)")

0.0001982506713799893

In [21]:
for var in all_variables(Nash_ipopt)
    println("$var -> $(value(var))")
end

P_G -> 0.99
P_B -> 0.011587040684504972
C_G -> 0.9956457740347624
C_B -> 0.8720262854818974
P_C -> 1.0014849157236008
M -> 0.9957935203422524
K -> 0.3763805114471349
GAMMA -> 0.011568680717540202
